<a href="https://colab.research.google.com/github/Rhysus85/MA5851_MasterClass1/blob/main/OceanInfinity_Visibility_HyperparameterTuning_PyScript.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This script looks at importing a model and tuning the hyperparameters to obtain the best architecture for the CNN.

In [ ]:
import cv2 
import numpy as np
from google.colab.patches import cv2_imshow #directed by google to reduce crashing
import pandas as pd
import csv
import math
from os.path import exists
import time

import matplotlib.pyplot as plt    # for plotting the images
from PIL import Image
from imutils import paths
from mpl_toolkits.axes_grid1 import ImageGrid

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Flatten, Input, MaxPooling2D, Activation, AveragePooling2D
from tensorflow import keras
from tensorflow.keras.models import Model

from tensorflow.keras.utils import load_img
from numpy import expand_dims

#from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import preprocess_input
import argparse



In [ ]:
#load libraries
#mount drive to access data 
import os 
from google.colab import drive

#I used my drive for the POC
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
#set directory
os.chdir('/content/drive/MyDrive/Ocean_Infinity')
os.listdir()

#output is:
#['Model',
# 'Output',
# 'Dataset',
# 'Raw Video',
# 'Raw Video Names.csv',
# 'Raw Dataset']


['Output',
 'Dataset',
 'Raw Video',
 'Raw Dataset',
 'POC',
 'image_metadata.csv',
 'hp_output',
 'my_dir',
 'Dataset 2 Class']

#THIS SECTION DOES HYPERPARAMETER TUNING

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [ ]:
output_dir = '/content/drive/MyDrive/Ocean_Infinity/Output/'
model_1_subfolder = 'Model_1/'
model = keras.models.load_model(output_dir+model_1_subfolder)

In [ ]:
!pip install keras-tuner --upgrade

In [ ]:
from kerastuner.tuners import Hyperband
from kerastuner.engine.hyperparameters import HyperParameters
import keras_tuner as kt
from tensorflow.keras.preprocessing.image import ImageDataGenerator


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [ ]:
training_local_path = 'Dataset/train/'
validation_local_path = 'Dataset/val/'

datagen = ImageDataGenerator(rescale=1.0/255.0)
img_width=224
img_height=224
batch_size = 50

train_batches = datagen.flow_from_directory(training_local_path
                                            ,class_mode='categorical'
                                            #,batch_size = batch_size
                                            ,target_size=(img_height, img_width))

val_batches = datagen.flow_from_directory(validation_local_path
                                          ,class_mode='categorical'
                                          #,batch_size = batch_size
                                          ,target_size=(img_height, img_width))

#to show the description when predicting
label_map = (train_batches.class_indices)
class_num = len(label_map)


Found 7070 images belonging to 5 classes.
Found 2050 images belonging to 5 classes.


In [ ]:
X_train = train_batches[0][0]
y_train = train_batches[0][1]
X_val = val_batches[0][0]
y_val = val_batches[0][1]

In [ ]:
print('X_train shape:', X_train.shape)
print('X_val shape:', X_val.shape)
print('y_train shape:', y_train.shape)
print('y_val shape:', y_val.shape)

X_train shape: (32, 224, 224, 3)
X_val shape: (32, 224, 224, 3)
y_train shape: (32, 5)
y_val shape: (32, 5)


In [ ]:
def model_builder(hp):
  hp_units = hp.Int('units', min_value=8, max_value=264, step=32)
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
  hp_activation = hp.Choice('activation', values=['relu','tanh'])
  hp_dropout = hp.Choice('dropout', values=[0.2, 0.3, 0.4, 0.5])
  #hp_pooling = hp.Choice('pooling',values=['MaxPooling2D','AveragePooling2D'])
 
  model = keras.Sequential()
  #input
  model.add(Conv2D(11,(3,3),input_shape=(224,224,3)))
  
  for i in range(hp.Int("n_layers",1,3)):
    model.add(Conv2D(hp.Int(f"layer_{i}_units", min_value=64, max_value=264, step=32),(3,3)))
    model.add(Activation(hp_activation))
    #model.add(AveragePooling2D(pool_size=(2,2)))

  model.add(Flatten())
  model.add(Dense(5))
  #model.add(Dropout(hp_dropout))
  model.add(Activation("softmax"))

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.categorical_crossentropy,#(from_logits=True),
                #loss="sparse_categorical_crossentropy",
                metrics=["accuracy"#,
                        #tf.keras.metrics.Precision(),
                        #tf.keras.metrics.Recall()
                        ]
                )

  return model


In [ ]:
output_dir = '/content/drive/MyDrive/Ocean_Infinity/Output/'
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     #executions_per_trial = 1,
                     #max_epochs=10,
                     factor=3,
                     directory=output_dir,
                     project_name='intro_to_kt_4')


In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)

tuner.search(x=X_train,
             y=y_train,
             epochs=10,
             #batch_size = batch,
             callbacks=[stop_early],
             validation_data = (X_val, y_val),
             validation_split=0.2        
             )



Trial 4 Complete [00h 00m 42s]
val_accuracy: 0.5

Best val_accuracy So Far: 0.5
Total elapsed time: 00h 06m 59s

Search: Running Trial #5

Value             |Best Value So Far |Hyperparameter
136               |72                |units
0.0001            |0.001             |learning_rate
tanh              |tanh              |activation
0.2               |0.2               |dropout
3                 |2                 |n_layers
224               |128               |layer_0_units
256               |64                |layer_1_units
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
4                 |4                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/2


1/1 [==============================] - ETA: 0s - loss: 1.6140 - accuracy: 0.2500

1/1 [==============================] - 161s 161s/step - loss: 1.6140 - accuracy: 0.2500 - val_loss: 8.1531 - val_accuracy: 0.3750
Epoch 2/2
1/1 [==============================] - ETA: 0s - loss: 5.8903 - accuracy: 0.5000

In [ ]:
tuner.get_best_hyperparameters()[0].values